In [1]:
import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp


# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C

In [2]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/atmout.xlsx")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['logreg', 'svm', 'sgd', 'dt', 'et', 'rf', 'gnb', 'mnb', 'bnb', 'gp', 'pa', 'knn', 'mlp', 'ada']", "fc")
df.methods = df.methods.replace("['rf', 'dt', 'svm']", "3c")
df.methods = df.methods.replace("['logreg', 'dt', 'knn']", "def")
df.methods = df.methods.replace("['svm']", "1c")
df.methods = df.methods.replace("['dt']", "1c")
df.methods = df.methods.replace("['rf']", "1c")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_binary = df[df.dataset.isin(fsIs_b) | df.dataset.isin(fsIl_b) | df.dataset.isin(flIs_b) | df.dataset.isin(flIl_b)]
df_multi = df[df.dataset.isin(fsIs_m) | df.dataset.isin(fsIl_m) | df.dataset.isin(flIs_m) | df.dataset.isin(flIl_m)]
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(873, 4)

In [3]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,873.0,0.798549,0.196157,0.006639,0.785519,0.811579


In [4]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
10,240,0.791272,0.187028,0.012073,0.767609,0.814934
30,337,0.806864,0.191000,0.010404,0.786471,0.827256
60,296,0.794982,0.209075,0.012152,0.771164,0.818800


In [5]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
1c,491,0.785143,0.201217,0.009081,0.767345,0.802941
3c,91,0.819211,0.195105,0.020453,0.779124,0.859298
def,109,0.816370,0.172775,0.016549,0.783934,0.848805
fc,182,0.813710,0.194678,0.014431,0.785426,0.841994


In [6]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score'].round(3)

N   Mean     SD     SE  95% Conf.  Interval
time_budget methods                                               
10          1c       124  0.770  0.199  0.018      0.735     0.805
            3c        16  0.804  0.178  0.044      0.717     0.892
            def       32  0.824  0.172  0.030      0.765     0.884
            fc        68  0.811  0.171  0.021      0.770     0.851
30          1c       202  0.795  0.201  0.014      0.767     0.822
            3c        34  0.834  0.150  0.026      0.783     0.885
            def       33  0.835  0.129  0.023      0.791     0.879
            fc        68  0.816  0.204  0.025      0.767     0.864
60          1c       165  0.785  0.204  0.016      0.753     0.816
            3c        41  0.813  0.234  0.037      0.741     0.884
            def       44  0.797  0.201  0.030      0.737     0.856
            fc        46  0.815  0.216  0.032      0.752     0.877

In [19]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 11, 861) =  0.690, p =  0.7487


In [20]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.6903
Date:                Fri, 01 Nov 2019   Prob (F-statistic):              0.749
Time:                        15:19:43   Log-Likelihood:                 187.58
No. Observations:                 873   AIC:                            -351.2
Df Residuals:                     861   BIC:                            -293.9
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  0.7703      0.018     43.643      0.000       0.736       0.805
C(time_budget)[T.30]                       0.0244      0.022      1.090      0.276      -0.020       0.068
C(time_budget)[T.60]                       0.0143      0.023      0.611      0.541      -0.032       0.060
C(methods)[T.3c]                           0.0342      0.052      0.655      0.513      -0.068       0.137
C(methods)[T.def]                          0.0542      0.039      1.391      0.165      -0.022       0.131
C(methods)[T.fc]                           0.0405      0.030      1.365      0.173      -0.018       0.099
C(time_budget)[T.30]:C(methods)[T.3c]      0.0051      0.064      0.080      0.936      -0.120       0.130
C(time_budget)[T.60]:C(methods)[T.3c]     -0.0061      0.062     -0.097      0.923      -0.129       0.117
C(time_budget)[T.30]:C(methods)[T.def]    -0.0142      0.054     -0.265      0.791      -0.120       0.091
C(time_budget)[T.60]:C(methods)[T.def]    -0.0421      0.051     -0.820      0.412      -0.143       0.059
C(time_budget)[T.30]:C(methods)[T.fc]     -0.0194      0.040     -0.479      0.632      -0.099       0.060
C(time_budget)[T.60]:C(methods)[T.fc]     -0.0102      0.044     -0.230      0.818      -0.097       0.077
==============================================================================
Omnibus:                      177.747   Durbin-Watson:                   2.178
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              304.289
Skew:                          -1.272   Prob(JB):                     8.41e-67
Kurtosis:                       4.376   Cond. No.                         17.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(time_budget),0.052076,2.0,0.674064,0.509902
C(methods),0.215844,3.0,1.862558,0.134346
C(time_budget):C(methods),0.037699,6.0,0.162657,0.986445
Residual,33.259222,861.0,NaN,NaN


In [22]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 5, 867) =  1.331, p =  0.2487


In [23]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.331
Date:                Fri, 01 Nov 2019   Prob (F-statistic):              0.249
Time:                        15:20:04   Log-Likelihood:                 187.08
No. Observations:                 873   AIC:                            -362.2
Df Residuals:                     867   BIC:                            -333.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.7762      0.014     54.391      0.000       0.748       0.804
C(time_budget)[T.30]     0.0180      0.017      1.082      0.280      -0.015       0.051
C(time_budget)[T.60]     0.0045      0.017      0.264      0.792      -0.029       0.038
C(methods)[T.3c]         0.0342      0.022      1.526      0.127      -0.010       0.078
C(methods)[T.def]        0.0329      0.021      1.580      0.114      -0.008       0.074
C(methods)[T.fc]         0.0296      0.017      1.732      0.084      -0.004       0.063
==============================================================================
Omnibus:                      179.394   Durbin-Watson:                   2.175
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              309.261
Skew:                          -1.278   Prob(JB):                     7.00e-68
Kurtosis:                       4.405   Cond. No.                         4.58
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.052076,2.0,0.677993,0.507904
C(methods),0.215844,3.0,1.873414,0.132473
Residual,33.296921,867.0,NaN,NaN


In [25]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.052076,0.026038,2.0,0.677993,0.507904,0.001552,-0.000736
C(methods),0.215844,0.071948,3.0,1.873414,0.132473,0.006431,0.002995
Residual,33.296921,0.038405,867.0,NaN,NaN,NaN,NaN


In [26]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c   0.0341 0.4251 -0.0235 0.0916  False
    1c    def   0.0312 0.4361 -0.0222 0.0846  False
    1c     fc   0.0286 0.3349 -0.0152 0.0723  False
    3c    def  -0.0028    0.9 -0.0744 0.0688  False
    3c     fc  -0.0055    0.9 -0.0702 0.0592  False
   def     fc  -0.0027    0.9 -0.0637 0.0584  False
---------------------------------------------------


In [27]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['time_budget'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    10     30   0.0156 0.6057 -0.0233 0.0545  False
    10     60   0.0037    0.9 -0.0363 0.0437  False
    30     60  -0.0119  0.709 -0.0486 0.0248  False
---------------------------------------------------


# Analysis <font color=blue>without </font> combining RF, DT and SVM into 1C

In [28]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/atmout.xlsx")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['logreg', 'svm', 'sgd', 'dt', 'et', 'rf', 'gnb', 'mnb', 'bnb', 'gp', 'pa', 'knn', 'mlp', 'ada']", "fc")
df.methods = df.methods.replace("['rf', 'dt', 'svm']", "3c")
df.methods = df.methods.replace("['logreg', 'dt', 'knn']", "def")
df.methods = df.methods.replace("['svm']", "1c")
df.methods = df.methods.replace("['dt']", "1c")
df.methods = df.methods.replace("['rf']", "1c")
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(873, 4)

In [29]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,873.0,0.798549,0.196157,0.006639,0.785519,0.811579


In [30]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
10,240,0.791272,0.187028,0.012073,0.767609,0.814934
30,337,0.806864,0.191000,0.010404,0.786471,0.827256
60,296,0.794982,0.209075,0.012152,0.771164,0.818800


In [31]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
1c,491,0.785143,0.201217,0.009081,0.767345,0.802941
3c,91,0.819211,0.195105,0.020453,0.779124,0.859298
def,109,0.816370,0.172775,0.016549,0.783934,0.848805
fc,182,0.813710,0.194678,0.014431,0.785426,0.841994


In [32]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

N      Mean        SD        SE  95% Conf.  Interval
time_budget methods                                                        
10          1c       124  0.770293  0.199364  0.017903   0.735202  0.805383
            3c        16  0.804479  0.177959  0.044490   0.717279  0.891678
            def       32  0.824496  0.171653  0.030344   0.765021  0.883970
            fc        68  0.810785  0.171226  0.020764   0.770087  0.851483
30          1c       202  0.794726  0.200704  0.014121   0.767048  0.822404
            3c        34  0.833986  0.150332  0.025782   0.783454  0.884518
            def       33  0.834716  0.129424  0.022530   0.790557  0.878874
            fc        68  0.815844  0.204182  0.024761   0.767312  0.864375
60          1c       165  0.784572  0.203745  0.015862   0.753483  0.815660
            3c        41  0.812707  0.234119  0.036563   0.741043  0.884371
            def       44  0.796701  0.201405  0.030363   0.737189  0.856212
            fc        46  0.814881  0.216091  0.031861   0.752434  0.877328

In [33]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 11, 861) =  0.690, p =  0.7487


In [34]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.6903
Date:                Fri, 01 Nov 2019   Prob (F-statistic):              0.749
Time:                        15:22:34   Log-Likelihood:                 187.58
No. Observations:                 873   AIC:                            -351.2
Df Residuals:                     861   BIC:                            -293.9
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  0.7703      0.018     43.643      0.000       0.736       0.805
C(time_budget)[T.30]                       0.0244      0.022      1.090      0.276      -0.020       0.068
C(time_budget)[T.60]                       0.0143      0.023      0.611      0.541      -0.032       0.060
C(methods)[T.3c]                           0.0342      0.052      0.655      0.513      -0.068       0.137
C(methods)[T.def]                          0.0542      0.039      1.391      0.165      -0.022       0.131
C(methods)[T.fc]                           0.0405      0.030      1.365      0.173      -0.018       0.099
C(time_budget)[T.30]:C(methods)[T.3c]      0.0051      0.064      0.080      0.936      -0.120       0.130
C(time_budget)[T.60]:C(methods)[T.3c]     -0.0061      0.062     -0.097      0.923      -0.129       0.117
C(time_budget)[T.30]:C(methods)[T.def]    -0.0142      0.054     -0.265      0.791      -0.120       0.091
C(time_budget)[T.60]:C(methods)[T.def]    -0.0421      0.051     -0.820      0.412      -0.143       0.059
C(time_budget)[T.30]:C(methods)[T.fc]     -0.0194      0.040     -0.479      0.632      -0.099       0.060
C(time_budget)[T.60]:C(methods)[T.fc]     -0.0102      0.044     -0.230      0.818      -0.097       0.077
==============================================================================
Omnibus:                      177.747   Durbin-Watson:                   2.178
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              304.289
Skew:                          -1.272   Prob(JB):                     8.41e-67
Kurtosis:                       4.376   Cond. No.                         17.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(time_budget),0.052076,2.0,0.674064,0.509902
C(methods),0.215844,3.0,1.862558,0.134346
C(time_budget):C(methods),0.037699,6.0,0.162657,0.986445
Residual,33.259222,861.0,NaN,NaN


In [36]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 5, 867) =  1.331, p =  0.2487


In [37]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.331
Date:                Fri, 01 Nov 2019   Prob (F-statistic):              0.249
Time:                        15:22:42   Log-Likelihood:                 187.08
No. Observations:                 873   AIC:                            -362.2
Df Residuals:                     867   BIC:                            -333.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.7762      0.014     54.391      0.000       0.748       0.804
C(time_budget)[T.30]     0.0180      0.017      1.082      0.280      -0.015       0.051
C(time_budget)[T.60]     0.0045      0.017      0.264      0.792      -0.029       0.038
C(methods)[T.3c]         0.0342      0.022      1.526      0.127      -0.010       0.078
C(methods)[T.def]        0.0329      0.021      1.580      0.114      -0.008       0.074
C(methods)[T.fc]         0.0296      0.017      1.732      0.084      -0.004       0.063
==============================================================================
Omnibus:                      179.394   Durbin-Watson:                   2.175
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              309.261
Skew:                          -1.278   Prob(JB):                     7.00e-68
Kurtosis:                       4.405   Cond. No.                         4.58
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [38]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.052076,2.0,0.677993,0.507904
C(methods),0.215844,3.0,1.873414,0.132473
Residual,33.296921,867.0,NaN,NaN


In [39]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.052076,0.026038,2.0,0.677993,0.507904,0.001552,-0.000736
C(methods),0.215844,0.071948,3.0,1.873414,0.132473,0.006431,0.002995
Residual,33.296921,0.038405,867.0,NaN,NaN,NaN,NaN


In [40]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c   0.0341 0.4251 -0.0235 0.0916  False
    1c    def   0.0312 0.4361 -0.0222 0.0846  False
    1c     fc   0.0286 0.3349 -0.0152 0.0723  False
    3c    def  -0.0028    0.9 -0.0744 0.0688  False
    3c     fc  -0.0055    0.9 -0.0702 0.0592  False
   def     fc  -0.0027    0.9 -0.0637 0.0584  False
---------------------------------------------------


# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C 
# Comparing FC (60 min) with 3C(30)

In [95]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "1c")
df.methods = df.methods.replace("['decision_tree']", "1c")
df.methods = df.methods.replace("['random_forest']", "1c")
df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.drop(df[(df.methods=='1c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(300, 3)

In [96]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,300.0,0.781032,0.201683,0.011644,0.758117,0.803947


In [97]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
30,240,0.771850,0.210135,0.013564,0.745264,0.798436
60,60,0.817759,0.159848,0.020636,0.777312,0.858206


In [98]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
1c,180,0.759652,0.222612,0.016593,0.727130,0.792173
3c,60,0.808447,0.163428,0.021098,0.767094,0.849799
fc,60,0.817759,0.159848,0.020636,0.777312,0.858206


In [99]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

N      Mean        SD        SE  95% Conf.  Interval
time_budget methods                                                        
30          1c       180  0.759652  0.222612  0.016593   0.727130  0.792173
            3c        60  0.808447  0.163428  0.021098   0.767094  0.849799
60          fc        60  0.817759  0.159848  0.020636   0.777312  0.858206

In [100]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 2, 297) =  2.588, p =  0.0769


In [101]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     2.588
Date:                Wed, 23 Oct 2019   Prob (F-statistic):             0.0769
Time:                        15:04:11   Log-Likelihood:                 57.728
No. Observations:                 300   AIC:                            -109.5
Df Residuals:                     297   BIC:                            -98.34
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
Intercept                                 0.7597      0.015     50.801      0.000       0.730       0.789
C(time_budget)[T.60]                      0.0194      0.010      1.943      0.053      -0.000       0.039
C(methods)[T.3c]                          0.0488      0.030      1.632      0.104      -0.010       0.108
C(methods)[T.fc]                          0.0194      0.010      1.943      0.053      -0.000       0.039
C(time_budget)[T.60]:C(methods)[T.3c]          0          0        nan        nan           0           0
C(time_budget)[T.60]:C(methods)[T.fc]     0.0194      0.010      1.943      0.053      -0.000       0.039
==============================================================================
Omnibus:                       74.811   Durbin-Watson:                   0.470
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              138.633
Skew:                          -1.338   Prob(JB):                     7.88e-31
Kurtosis:                       4.982   Cond. No.                          inf
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [102]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\statsmodels\base\model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\statsmodels\base\model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\statsmodels\base\model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


,sum_sq,df,F,PR(>F)
C(time_budget),NaN,1.0,NaN,NaN
C(methods),3.370358e-17,2.0,4.186913e-16,1.000000
C(time_budget):C(methods),3.038849e-01,2.0,3.775087e+00,0.052966
Residual,1.195387e+01,297.0,NaN,NaN


In [103]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 2, 297) =  2.588, p =  0.0769


In [81]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     3.765
Date:                Wed, 23 Oct 2019   Prob (F-statistic):            0.00488
Time:                        14:45:28   Log-Likelihood:                 49.945
No. Observations:                 660   AIC:                            -89.89
Df Residuals:                     655   BIC:                            -67.43
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.7108      0.017     42.349      0.000       0.678       0.744
C(time_budget)[T.30]     0.0488      0.024      2.058      0.040       0.002       0.095
C(time_budget)[T.60]     0.0417      0.024      1.756      0.080      -0.005       0.088
C(methods)[T.3c]         0.0488      0.034      1.454      0.147      -0.017       0.115
C(methods)[T.fc]         0.0653      0.034      1.945      0.052      -0.001       0.131
==============================================================================
Omnibus:                      156.110   Durbin-Watson:                   0.424
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              294.256
Skew:                          -1.371   Prob(JB):                     1.27e-64
Kurtosis:                       4.782   Cond. No.                         4.79
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [82]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.250448,2.0,2.469388,0.085424
C(methods),0.298902,2.0,2.947136,0.053186
Residual,33.215397,655.0,NaN,NaN


In [83]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.250448,0.125224,2.0,2.469388,0.085424,0.007417,0.004407
C(methods),0.298902,0.149451,2.0,2.947136,0.053186,0.008852,0.005840
Residual,33.215397,0.050711,655.0,NaN,NaN,NaN,NaN


In [84]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  1C     3c    0.0675  -0.0047 0.1396 False 
  1C     fc    0.0768   0.0046 0.1489  True 
  3c     fc    0.0093  -0.0875 0.1061 False 
--------------------------------------------


# Wilcoxon signed-rank test

In [87]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svm")
df.methods = df.methods.replace("['decision_tree']", "DT")
df.methods = df.methods.replace("['random_forest']", "RF")
df.shape

(900, 3)

In [89]:
# Wilcoxon signed-rank test
from numpy.random import seed
from numpy.random import randn
from scipy.stats import wilcoxon
# seed the random number generator
seed(1)
# generate two independent samples
data1 = df.f1score[df.methods=='3c']
data2 = df.f1score[df.methods=='fc']
# compare samples
stat, p = wilcoxon(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Same distribution (fail to reject H0)')
else:
    print('Different distribution (reject H0)')

Statistics=5989.500, p=0.006
Different distribution (reject H0)


In [92]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svm")
df.methods = df.methods.replace("['decision_tree']", "DT")
df.methods = df.methods.replace("['random_forest']", "RF")
df.shape

(900, 3)

In [94]:
# Wilcoxon signed-rank test
from numpy.random import seed
from numpy.random import randn
from scipy.stats import wilcoxon
# seed the random number generator
seed(1)
# generate two independent samples
data1 = df.f1score[df.methods=='3c']
data2 = df.f1score[df.methods=='fc']
# compare samples
stat, p = wilcoxon(data1, data2)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Same distribution (fail to reject H0)')
else:
    print('Different distribution (reject H0)')

Statistics=5989.500, p=0.006
Different distribution (reject H0)
